In [5]:
import cv2
import numpy as np
from itertools import count
from skimage.measure import ransac
from skimage.transform import FundamentalMatrixTransform, EssentialMatrixTransform
import matplotlib.pyplot as plt

vid_path = '5.hevc'
w = 1164
h = 874
F = 910
#gt = np.loadtxt('0.txt')

def poseRt(R, t):
  ret = np.eye(4)
  ret[:3, :3] = R
  ret[:3, 3] = t
  return ret

K = np.array([
    [F, 0, w//2],
    [0, F, h//2],
    [0, 0, 1]])
K_inv = np.linalg.inv(K)
W = np.mat([[0,-1,0],[1,0,0],[0,0,1]],dtype=float)

def normalize(x):
  x = add_third(x)
  return (K_inv @ x.T).T[:, 0:2]
def denormalize(x):
  return (K @ x)[:-1]

def add_third(x):
  return np.pad(x, pad_width = ((0,0), (0,1)) ,constant_values=1)
def remove_third(x):
  return x[:,[0,1]]

def generate_frames(vid_path):
    video = cv2.VideoCapture(vid_path, cv2.CAP_FFMPEG)
    _, prev_frame = video.read()
    for t in count():
      ret, curr_frame = video.read()
      if not ret:
        break
      yield prev_frame, curr_frame
      prev_frame = curr_frame
    video.release()
    cv2.destroyAllWindows()

def extractFeatures(frame):
  orb = cv2.SIFT_create()
  # only works on b/w images
  #pts = cv2.goodFeaturesToTrack(np.mean(frame,axis=2).astype(np.uint8), 3000, 0.05, minDistance=7)

  pts = orb.detect(frame, None)
  # we need kp class to feed it into ORB.compute to get descriptors
  #kps = [cv2.KeyPoint(x=f[0][0], y=f[0][1], size=10) for f in pts]
  kps = pts
  kps, des = orb.compute(frame, kps)
  return np.array([(kp.pt[0], kp.pt[1]) for kp in kps]), des

def bfmatcher(kps, dess):
  bf = cv2.BFMatcher(cv2.NORM_L1)
  # find closest descriptors between frames, hamming norm for ORB (BRIEF, BRISK...)
  # they are binary string types
  # L1, L2 for SIFT/SURF
  matches = bf.knnMatch(dess[0], dess[1], k=2)
  res = []
  # DMatch obj: distance (the lower the better)
  # trainIdx: index of the descriptor in train desc
  # queryIdx: index of the descriptor in query desc
  # imgIdx: index of the train image
  for m,n in matches:
    # Lowe's ratio test
    # https://stackoverflow.com/questions/51197091/how-does-the-lowes-ratio-test-work
    if m.distance < 0.70 * n.distance:
        kp1 = kps[0][m.queryIdx]
        kp2 = kps[1][m.trainIdx]
        res.append((kp1, kp2))
  res = np.array(res)
  # prune the outliers by fitting
  assert len(res)>=8, 'not enough points'
  model, inliers = ransac((normalize(res[:,0]), normalize(res[:,1])),
                          #FundamentalMatrixTransform, 
                          EssentialMatrixTransform,
                          min_samples=8,
                          residual_threshold=0.001, 
                          max_trials=1000)

  U, D, Vt = np.linalg.svd(model.params)

  if np.linalg.det(U) < 0:
    U *= -1.0
  if np.linalg.det(Vt) < 0:
    Vt *= -1.0
  R1 = U @ W @ Vt
  R2 = U @ W.T @ Vt
  if abs(np.sum(R1.diagonal())-3) > abs(np.sum(R2.diagonal())-3):
    R = R2
  else: 
    R = R1
  t = U[:, 2]
  if t[2] < 0:
    t *= -1
  #pose = poseRt(R, t)
  print (t)
  print (f'good kps: {len(res[inliers])/len(res)*100:.2f}% out of {len(res)}')
  return res[inliers,0], res[inliers,1]

In [6]:
pred = []

for i, (p, c) in enumerate(generate_frames(vid_path)):
  
  kps_1, des_1 = extractFeatures(p)
  kps_2, des_2 = extractFeatures(c)

  p1, p2 = bfmatcher([kps_1, kps_2], [des_1, des_2])

  for p in p2:
    cv2.circle(c, (int(p[0]), int(p[1])), 1, (255,255,0)) 
  
  for k1, k2 in zip(p1, p2):
    cv2.line(c, tuple(k1.astype(int)), tuple(k2.astype(int)), (0,200,200), 1)
  
  cv2.imshow('v', c)
  key = cv2.waitKey(1)
  if key == ord('q'):
    break

cv2.destroyAllWindows()

[0.0180855  0.00652894 0.99981513]
good kps: 64.92% out of 305
[ 0.04013526 -0.00753359  0.99916585]
good kps: 69.93% out of 153
[ 0.01182827 -0.02438366  0.9996327 ]
good kps: 80.92% out of 283
[ 0.01068061 -0.04334134  0.99900323]
good kps: 75.00% out of 140
[ 0.04484933 -0.01344429  0.99890329]
good kps: 85.12% out of 215
[ 0.00450344 -0.01054701  0.99993424]
good kps: 82.98% out of 141
[-0.03226346  0.01163945  0.99941162]
good kps: 87.67% out of 227
[-0.03254857 -0.01640232  0.99933556]
good kps: 79.35% out of 155
[-0.34110229  0.10209394  0.93446565]
good kps: 85.04% out of 234
[ 0.01011662 -0.00229175  0.9999462 ]
good kps: 77.91% out of 172
[-0.19085415 -0.00933514  0.98157402]
good kps: 86.03% out of 179
[ 0.94242451 -0.10162721  0.31860314]
good kps: 81.25% out of 144
[-0.05702801 -0.00623773  0.99835309]
good kps: 89.39% out of 198
[-0.05962691  0.00277482  0.99821688]
good kps: 90.37% out of 135
[-0.22669533  0.00997159  0.97391468]
good kps: 87.46% out of 279
[-0.29497716 